# London House Price Data
100,000 sold house prices from 1995 to Oct 2024

https://www.kaggle.com/datasets/jakewright/house-price-data
https://www.kaggle.com/code/jakewright/predicting-house-prices

In [ ]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import optuna
import numpy as np
from sklearn.metrics import mean_absolute_error
import warnings

pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
all_properties = pd.read_parquet('/kaggle/input/house-price-data/kaggle_london_house_price_data.parquet').reset_index(drop=True)

In [ ]:
all_properties['history_date'] = pd.to_datetime(all_properties['history_date'])

all_properties['week'] = all_properties['history_date'].dt.isocalendar().week
all_properties['dayofyear'] = all_properties['history_date'].dt.day_of_year
all_properties['dayofmonth'] = all_properties['history_date'].dt.day

all_properties['month'] = all_properties['history_date'].dt.month
all_properties['year'] = all_properties['history_date'].dt.year
all_properties['quarter'] = all_properties['history_date'].dt.quarter

all_properties['bed-bath'] = all_properties['bedrooms'].astype(str) +'-'+ all_properties['bathrooms'].astype(str)
all_properties['bed-liv'] = all_properties['bedrooms'].astype(str) +'-'+ all_properties['livingRooms'].astype(str)
all_properties['bed-bath-liv'] = all_properties['bedrooms'].astype(str) +'-'+ all_properties['bathrooms'].astype(str) +'-'+ all_properties['livingRooms'].astype(str)

all_properties['Area_Binned_Low_500'] = pd.cut(all_properties['floorAreaSqM'], bins=500).apply(lambda x: x.left if pd.notnull(x) else None)
all_properties['Area_Binned_Low_250'] = pd.cut(all_properties['floorAreaSqM'], bins=250).apply(lambda x: x.left if pd.notnull(x) else None)
all_properties['Area_Binned_Low_10'] = pd.cut(all_properties['floorAreaSqM'], bins=10).apply(lambda x: x.left if pd.notnull(x) else None)

all_properties['quadrant'] = all_properties['postcode'].str[:1]
all_properties['total_room_count'] = all_properties[['bedrooms', 'bathrooms', 'livingRooms']].sum(axis=1)
all_properties['house_type'] = all_properties['tenure'] + '-' + all_properties['propertyType']

categorical_columns = ['bed-bath', 'bed-liv', 'bed-bath-liv', 'postcode', 'outcode', 'bathrooms', 'bedrooms', 'livingRooms', 'tenure', 'propertyType', 'currentEnergyRating', 'saleEstimate_confidenceLevel', 'quadrant', 'Area_Binned_Low_500','Area_Binned_Low_250', 'Area_Binned_Low_10', 'total_room_count', 'house_type']
needed_columns = categorical_columns + ['floorAreaSqM', 'latitude', 'longitude', 'history_price', 'month', 'quarter', 'year', 'week', 'dayofmonth', 'dayofyear']
df = all_properties.sort_values(by=['history_date'])[needed_columns]
outcode_count = df.groupby(by=['outcode']).size().sort_values().reset_index()
df = df[df['outcode'].isin(outcode_count[outcode_count[0]>400]['outcode'].tolist())]

df = df.drop_duplicates().reset_index(drop=True)

df['average_sq_per_room'] = (df['floorAreaSqM'] / df['total_room_count']).replace({np.inf:np.nan})
df['average_sq_per_bed_livingrooms'] = (df['floorAreaSqM'] / df[['bedrooms', 'livingRooms']].sum(axis=1)).replace({np.inf:np.nan})
df['average_sq_per_bed_bathrooms'] = (df['floorAreaSqM'] / df[['bedrooms', 'bathrooms']].sum(axis=1)).replace({np.inf:np.nan})

In [ ]:
for col in categorical_columns:
    df[col] = df[col].astype('category')

df = df.dropna(subset=['history_price'])
df['history_log_price'] = np.log(df['history_price'])
test_size=0.05

test_size = int(df.shape[0]*test_size)
final_test_size = int(test_size*0.3)

train_set = df.head(-test_size)
test_set = df.tail(test_size)


for col in ['outcode']:
    total_mean_outcode_average = train_set.groupby(by=col)['history_price'].mean().reset_index().rename(columns={'history_price':f'total_average_{col}_price'})
    total_median_outcode_average = train_set.groupby(by=col)['history_price'].median().reset_index().rename(columns={'history_price':f'total_median_{col}_price'})
    total_mean_outcode_average_log = train_set.groupby(by=col)['history_log_price'].mean().reset_index().rename(columns={'history_log_price':f'total_average_{col}_log_price'})
    total_median_outcode_average_log = train_set.groupby(by=col)['history_log_price'].median().reset_index().rename(columns={'history_log_price':f'total_median_{col}_log_price'})
    
    train_set = train_set.merge(total_mean_outcode_average, how='outer', on=col)
    train_set = train_set.merge(total_median_outcode_average, how='outer', on=col)
    train_set = train_set.merge(total_mean_outcode_average_log, how='outer', on=col)
    train_set = train_set.merge(total_median_outcode_average_log, how='outer', on=col)
    
    test_set = test_set.merge(total_mean_outcode_average, how='outer', on=col)
    test_set = test_set.merge(total_median_outcode_average, how='outer', on=col)
    test_set = test_set.merge(total_mean_outcode_average_log, how='outer', on=col)
    test_set = test_set.merge(total_median_outcode_average_log, how='outer', on=col)
    
    for period in ['month', 'year', 'week', 'dayofmonth', 'dayofyear']:
        mean_outcode_average = train_set.groupby(by=[col, period])['history_price'].mean().reset_index().rename(columns={'history_price':f'average_{col}_price_wrt_{period}'})
        median_outcode_average = train_set.groupby(by=[col, period])['history_price'].median().reset_index().rename(columns={'history_price':f'median_{col}_price_wrt_{period}'})
        mean_outcode_average_log = train_set.groupby(by=[col, period])['history_log_price'].mean().reset_index().rename(columns={'history_log_price':f'average_{col}_log_price_wrt_{period}'})
        median_outcode_average_log = train_set.groupby(by=[col, period])['history_log_price'].median().reset_index().rename(columns={'history_log_price':f'median_{col}_log_price_wrt_{period}'})


        train_set = train_set.merge(mean_outcode_average, how='outer', on=[col, period])
        train_set = train_set.merge(median_outcode_average, how='outer', on=[col, period])
        train_set = train_set.merge(mean_outcode_average_log, how='outer', on=[col, period])
        train_set = train_set.merge(median_outcode_average_log, how='outer', on=[col, period])

        test_set = test_set.merge(mean_outcode_average, how='outer', on=[col, period])
        test_set = test_set.merge(median_outcode_average, how='outer', on=[col, period])
        test_set = test_set.merge(mean_outcode_average_log, how='outer', on=[col, period])
        test_set = test_set.merge(median_outcode_average_log, how='outer', on=[col, period])

train_set = train_set.dropna(subset=['history_price'])
test_set = test_set.dropna(subset=['history_price'])

final_test_set = test_set.tail(final_test_size)
test_set = test_set.head(-final_test_size)

X_train, y_train = train_set.drop(columns=['history_price', 'history_log_price']),  train_set['history_price']
X_test, y_test = test_set.drop(columns=['history_price', 'history_log_price']),  test_set['history_price']

X_test_final, y_test_final = final_test_set.drop(columns=['history_price', 'history_log_price']), final_test_set['history_price']

lgb_model = lgb.LGBMRegressor(verbose=-2, subsample=0.8, colsample_bytree=0.8, linear_tree=False)
xgb_model = xgb.XGBRegressor(enable_categorical=True, colsample_bytree=0.8)

lgb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

week_columns = [col for col in X_train.columns if 'week' in col]
dayofmonth_columns = [col for col in X_train.columns if 'dayofmonth' in col]
dayofyear_columns = [col for col in X_train.columns if 'dayofyear' in col]

month_columns = [col for col in X_train.columns if 'month' in col and 'dayofmonth' not in col]
year_columns = [col for col in X_train.columns if 'year' in col and 'dayofyear' not in col]

sq_ft_columns = ['average_sq_per_room', 'average_sq_per_bed_livingrooms', 'average_sq_per_bed_bathrooms']
bed_bathroom_columns = ['bed-bath-liv', 'bed-liv', 'bed-bath']
binned_columns = ['Area_Binned_Low_500', 'Area_Binned_Low_250', 'Area_Binned_Low_10']
mean_absolute_error(y_pred=lgb_model.predict(X_test), y_true=y_test)
181832.34616247966
mean_absolute_error(y_pred=xgb_model.predict(X_test), y_true=y_test)
186726.1618394154

In [ ]:
for col in ['outcode']:
    total_mean_outcode_average = train_set.groupby(by=col)['history_price'].mean().reset_index().rename(columns={'history_price':f'total_average_{col}_price'})
    total_median_outcode_average = train_set.groupby(by=col)['history_price'].median().reset_index().rename(columns={'history_price':f'total_median_{col}_price'})
    total_mean_outcode_average_log = train_set.groupby(by=col)['history_log_price'].mean().reset_index().rename(columns={'history_log_price':f'total_average_{col}_log_price'})
    total_median_outcode_average_log = train_set.groupby(by=col)['history_log_price'].median().reset_index().rename(columns={'history_log_price':f'total_median_{col}_log_price'})
    
    train_set = train_set.merge(total_mean_outcode_average, how='outer', on=col)
    train_set = train_set.merge(total_median_outcode_average, how='outer', on=col)
    train_set = train_set.merge(total_mean_outcode_average_log, how='outer', on=col)
    train_set = train_set.merge(total_median_outcode_average_log, how='outer', on=col)
    
    test_set = test_set.merge(total_mean_outcode_average, how='outer', on=col)
    test_set = test_set.merge(total_median_outcode_average, how='outer', on=col)
    test_set = test_set.merge(total_mean_outcode_average_log, how='outer', on=col)
    test_set = test_set.merge(total_median_outcode_average_log, how='outer', on=col)
    
    for period in ['month', 'year', 'week', 'dayofmonth', 'dayofyear']:
        mean_outcode_average = train_set.groupby(by=[col, period])['history_price'].mean().reset_index().rename(columns={'history_price':f'average_{col}_price_wrt_{period}'})
        median_outcode_average = train_set.groupby(by=[col, period])['history_price'].median().reset_index().rename(columns={'history_price':f'median_{col}_price_wrt_{period}'})
        mean_outcode_average_log = train_set.groupby(by=[col, period])['history_log_price'].mean().reset_index().rename(columns={'history_log_price':f'average_{col}_log_price_wrt_{period}'})
        median_outcode_average_log = train_set.groupby(by=[col, period])['history_log_price'].median().reset_index().rename(columns={'history_log_price':f'median_{col}_log_price_wrt_{period}'})


        train_set = train_set.merge(mean_outcode_average, how='outer', on=[col, period])
        train_set = train_set.merge(median_outcode_average, how='outer', on=[col, period])
        train_set = train_set.merge(mean_outcode_average_log, how='outer', on=[col, period])
        train_set = train_set.merge(median_outcode_average_log, how='outer', on=[col, period])

        test_set = test_set.merge(mean_outcode_average, how='outer', on=[col, period])
        test_set = test_set.merge(median_outcode_average, how='outer', on=[col, period])
        test_set = test_set.merge(mean_outcode_average_log, how='outer', on=[col, period])
        test_set = test_set.merge(median_outcode_average_log, how='outer', on=[col, period])

train_set = train_set.dropna(subset=['history_price'])
test_set = test_set.dropna(subset=['history_price'])

final_test_set = test_set.tail(final_test_size)
test_set = test_set.head(-final_test_size)

X_train, y_train = train_set.drop(columns=['history_price', 'history_log_price']),  train_set['history_price']
X_test, y_test = test_set.drop(columns=['history_price', 'history_log_price']),  test_set['history_price']

X_test_final, y_test_final = final_test_set.drop(columns=['history_price', 'history_log_price']), final_test_set['history_price']

lgb_model = lgb.LGBMRegressor(verbose=-2, subsample=0.8, colsample_bytree=0.8, linear_tree=False)
xgb_model = xgb.XGBRegressor(enable_categorical=True, colsample_bytree=0.8)

lgb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

week_columns = [col for col in X_train.columns if 'week' in col]
dayofmonth_columns = [col for col in X_train.columns if 'dayofmonth' in col]
dayofyear_columns = [col for col in X_train.columns if 'dayofyear' in col]

month_columns = [col for col in X_train.columns if 'month' in col and 'dayofmonth' not in col]
year_columns = [col for col in X_train.columns if 'year' in col and 'dayofyear' not in col]

sq_ft_columns = ['average_sq_per_room', 'average_sq_per_bed_livingrooms', 'average_sq_per_bed_bathrooms']
bed_bathroom_columns = ['bed-bath-liv', 'bed-liv', 'bed-bath']
binned_columns = ['Area_Binned_Low_500', 'Area_Binned_Low_250', 'Area_Binned_Low_10']
mean_absolute_error(y_pred=lgb_model.predict(X_test), y_true=y_test)
181832.34616247966
mean_absolute_error(y_pred=xgb_model.predict(X_test), y_true=y_test)
186726.1618394154

In [ ]:
ll_properties = pd.read_parquet('/kaggle/input/house-price-data/kaggle_london_house_price_data.parquet').reset_index(drop=True)
all_properties['history_date'] = pd.to_datetime(all_properties['history_date'])

all_properties['week'] = all_properties['history_date'].dt.isocalendar().week
all_properties['dayofyear'] = all_properties['history_date'].dt.day_of_year
all_properties['dayofmonth'] = all_properties['history_date'].dt.day

all_properties['month'] = all_properties['history_date'].dt.month
all_properties['year'] = all_properties['history_date'].dt.year
all_properties['quarter'] = all_properties['history_date'].dt.quarter

all_properties['bed-bath'] = all_properties['bedrooms'].astype(str) +'-'+ all_properties['bathrooms'].astype(str)
all_properties['bed-liv'] = all_properties['bedrooms'].astype(str) +'-'+ all_properties['livingRooms'].astype(str)
all_properties['bed-bath-liv'] = all_properties['bedrooms'].astype(str) +'-'+ all_properties['bathrooms'].astype(str) +'-'+ all_properties['livingRooms'].astype(str)

all_properties['Area_Binned_Low_500'] = pd.cut(all_properties['floorAreaSqM'], bins=500).apply(lambda x: x.left if pd.notnull(x) else None)
all_properties['Area_Binned_Low_250'] = pd.cut(all_properties['floorAreaSqM'], bins=250).apply(lambda x: x.left if pd.notnull(x) else None)
all_properties['Area_Binned_Low_10'] = pd.cut(all_properties['floorAreaSqM'], bins=10).apply(lambda x: x.left if pd.notnull(x) else None)

all_properties['quadrant'] = all_properties['postcode'].str[:1]
all_properties['total_room_count'] = all_properties[['bedrooms', 'bathrooms', 'livingRooms']].sum(axis=1)
all_properties['house_type'] = all_properties['tenure'] + '-' + all_properties['propertyType']

categorical_columns = ['bed-bath', 'bed-liv', 'bed-bath-liv', 'postcode', 'outcode', 'bathrooms', 'bedrooms', 'livingRooms', 'tenure', 'propertyType', 'currentEnergyRating', 'saleEstimate_confidenceLevel', 'quadrant', 'Area_Binned_Low_500','Area_Binned_Low_250', 'Area_Binned_Low_10', 'total_room_count', 'house_type']
needed_columns = categorical_columns + ['floorAreaSqM', 'latitude', 'longitude', 'history_price', 'month', 'quarter', 'year', 'week', 'dayofmonth', 'dayofyear']
df = all_properties.sort_values(by=['history_date'])[needed_columns]
outcode_count = df.groupby(by=['outcode']).size().sort_values().reset_index()
df = df[df['outcode'].isin(outcode_count[outcode_count[0]>400]['outcode'].tolist())]

df = df.drop_duplicates().reset_index(drop=True)

df['average_sq_per_room'] = (df['floorAreaSqM'] / df['total_room_count']).replace({np.inf:np.nan})
df['average_sq_per_bed_livingrooms'] = (df['floorAreaSqM'] / df[['bedrooms', 'livingRooms']].sum(axis=1)).replace({np.inf:np.nan})
df['average_sq_per_bed_bathrooms'] = (df['floorAreaSqM'] / df[['bedrooms', 'bathrooms']].sum(axis=1)).replace({np.inf:np.nan})
for col in categorical_columns:
    df[col] = df[col].astype('category')

df = df.dropna(subset=['history_price'])
df['history_log_price'] = np.log(df['history_price'])
test_size=0.05

test_size = int(df.shape[0]*test_size)
final_test_size = int(test_size*0.3)

train_set = df.head(-test_size)
test_set = df.tail(test_size)

In [ ]:
for col in ['outcode']:
    total_mean_outcode_average = train_set.groupby(by=col)['history_price'].mean().reset_index().rename(columns={'history_price':f'total_average_{col}_price'})
    total_median_outcode_average = train_set.groupby(by=col)['history_price'].median().reset_index().rename(columns={'history_price':f'total_median_{col}_price'})
    total_mean_outcode_average_log = train_set.groupby(by=col)['history_log_price'].mean().reset_index().rename(columns={'history_log_price':f'total_average_{col}_log_price'})
    total_median_outcode_average_log = train_set.groupby(by=col)['history_log_price'].median().reset_index().rename(columns={'history_log_price':f'total_median_{col}_log_price'})
    
    train_set = train_set.merge(total_mean_outcode_average, how='outer', on=col)
    train_set = train_set.merge(total_median_outcode_average, how='outer', on=col)
    train_set = train_set.merge(total_mean_outcode_average_log, how='outer', on=col)
    train_set = train_set.merge(total_median_outcode_average_log, how='outer', on=col)
    
    test_set = test_set.merge(total_mean_outcode_average, how='outer', on=col)
    test_set = test_set.merge(total_median_outcode_average, how='outer', on=col)
    test_set = test_set.merge(total_mean_outcode_average_log, how='outer', on=col)
    test_set = test_set.merge(total_median_outcode_average_log, how='outer', on=col)
    
    for period in ['month', 'year', 'week', 'dayofmonth', 'dayofyear']:
        mean_outcode_average = train_set.groupby(by=[col, period])['history_price'].mean().reset_index().rename(columns={'history_price':f'average_{col}_price_wrt_{period}'})
        median_outcode_average = train_set.groupby(by=[col, period])['history_price'].median().reset_index().rename(columns={'history_price':f'median_{col}_price_wrt_{period}'})
        mean_outcode_average_log = train_set.groupby(by=[col, period])['history_log_price'].mean().reset_index().rename(columns={'history_log_price':f'average_{col}_log_price_wrt_{period}'})
        median_outcode_average_log = train_set.groupby(by=[col, period])['history_log_price'].median().reset_index().rename(columns={'history_log_price':f'median_{col}_log_price_wrt_{period}'})


        train_set = train_set.merge(mean_outcode_average, how='outer', on=[col, period])
        train_set = train_set.merge(median_outcode_average, how='outer', on=[col, period])
        train_set = train_set.merge(mean_outcode_average_log, how='outer', on=[col, period])
        train_set = train_set.merge(median_outcode_average_log, how='outer', on=[col, period])

        test_set = test_set.merge(mean_outcode_average, how='outer', on=[col, period])
        test_set = test_set.merge(median_outcode_average, how='outer', on=[col, period])
        test_set = test_set.merge(mean_outcode_average_log, how='outer', on=[col, period])
        test_set = test_set.merge(median_outcode_average_log, how='outer', on=[col, period])

train_set = train_set.dropna(subset=['history_price'])
test_set = test_set.dropna(subset=['history_price'])

final_test_set = test_set.tail(final_test_size)
test_set = test_set.head(-final_test_size)

X_train, y_train = train_set.drop(columns=['history_price', 'history_log_price']),  train_set['history_price']
X_test, y_test = test_set.drop(columns=['history_price', 'history_log_price']),  test_set['history_price']

X_test_final, y_test_final = final_test_set.drop(columns=['history_price', 'history_log_price']), final_test_set['history_price']

lgb_model = lgb.LGBMRegressor(verbose=-2, subsample=0.8, colsample_bytree=0.8, linear_tree=False)
xgb_model = xgb.XGBRegressor(enable_categorical=True, colsample_bytree=0.8)

lgb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

week_columns = [col for col in X_train.columns if 'week' in col]
dayofmonth_columns = [col for col in X_train.columns if 'dayofmonth' in col]
dayofyear_columns = [col for col in X_train.columns if 'dayofyear' in col]

month_columns = [col for col in X_train.columns if 'month' in col and 'dayofmonth' not in col]
year_columns = [col for col in X_train.columns if 'year' in col and 'dayofyear' not in col]

sq_ft_columns = ['average_sq_per_room', 'average_sq_per_bed_livingrooms', 'average_sq_per_bed_bathrooms']
bed_bathroom_columns = ['bed-bath-liv', 'bed-liv', 'bed-bath']
binned_columns = ['Area_Binned_Low_500', 'Area_Binned_Low_250', 'Area_Binned_Low_10']

In [ ]:
mean_absolute_error(y_pred=lgb_model.predict(X_test), y_true=y_test)
# 181832.34616247966

In [ ]:
mean_absolute_error(y_pred=xgb_model.predict(X_test), y_true=y_test)
# 186726.1618394154